In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from data_process import TextEyeTrackingDataset
import numpy as np

# torch.manual_seed(seed)
# np.random.seed(seed)

# 加载数据
dataset = TextEyeTrackingDataset('data/not_skip_data.csv')
batch_size = 1

# 假设dataset是你已经加载的完整数据集
train_size = int(len(dataset) * 0.8)  # 以80%的数据作为训练集
test_size = len(dataset) - train_size  # 剩余20%的数据作为测试集
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(len(train_loader))
print(len(test_loader))

102
26


In [9]:
import torch.nn as nn
from model import MultiModalModel
from train_and_evaluate import train_model
import warnings
from sklearn import exceptions

# 忽略特定类型的警告
warnings.filterwarnings("ignore", category=exceptions.UndefinedMetricWarning)


hidden_dim = 128  # LSTM隐藏层维度
output_dim = 1  # 输出维度，二分类问题
learning_rate = 0.1

model = MultiModalModel(hidden_dim, output_dim).to(device)
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)

pos_weight = torch.tensor([30])  # 大于1的值，表示正类相对于负类的权重
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight).to(device)

tensorboard_dir = 'tensorlog/after/not-skip-data/'
# 训练模型
train_model(model, train_loader, test_loader, optimizer, criterion, device, tensorboard_dir, epochs=50)

-----Epoch 1-------
Train metric in Epoch 1, Loss: nan, Accuracy: 0.97, AUC: 0.5, Precision: 0.0, Recall: 0.0, F1-score: 0
Test metric in Epoch 1, Loss: nan, Accuracy: 0.98, AUC: 0.5, Precision: 0.0, Recall: 0.0, F1-score: 0
-----Epoch 2-------
Train metric in Epoch 2, Loss: nan, Accuracy: 0.99, AUC: 0.5, Precision: 0.0, Recall: 0.0, F1-score: 0
Test metric in Epoch 2, Loss: nan, Accuracy: 0.98, AUC: 0.5, Precision: 0.0, Recall: 0.0, F1-score: 0
-----Epoch 3-------
Train metric in Epoch 3, Loss: nan, Accuracy: 0.9, AUC: 0.5, Precision: 0.0, Recall: 0.0, F1-score: 0
Test metric in Epoch 3, Loss: nan, Accuracy: 0.98, AUC: 0.5, Precision: 0.0, Recall: 0.0, F1-score: 0
-----Epoch 4-------
Train metric in Epoch 4, Loss: nan, Accuracy: 1.0, AUC: 0.5, Precision: 0.0, Recall: 0.0, F1-score: 0
Test metric in Epoch 4, Loss: nan, Accuracy: 0.98, AUC: 0.5, Precision: 0.0, Recall: 0.0, F1-score: 0
-----Epoch 5-------
Train metric in Epoch 5, Loss: nan, Accuracy: 1.0, AUC: 0.5, Precision: 0.0, Recal

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x14c7723281d0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [ ]:
!tensorboard --logdir='tensorlog/all'